> [GNN-Dynamic](https://arxiv.org/abs/2210.08408) - it plans a trajectory by prioritizing the edge to explore based on a Graph Neural Network heuristic function in a dynamic environment.

In [1]:
cd ..

/Users/ruipeng/Desktop/lemp


In [2]:
from environment.dynamic.simple2arm_env import Simple2ArmEnv
from objects.dynamic_object import DynamicObject
from robot.abstract_robot import DynamicRobotFactory
from objects.trajectory import WaypointLinearTrajectory
from robot.simple2arm_robot import Simple2ArmRobot
from planner.sipp_planner import SippPlanner
from utils.utils import seed_everything

import numpy as np

pybullet build time: Mar 16 2022 13:26:26


In [ ]:
waypoints = np.linspace(0, np.pi/4, 40).tolist()
waypoints = [np.array([p,p]) for p in waypoints]
traj = WaypointLinearTrajectory(waypoints=waypoints)
robot = Simple2ArmRobot(base_position=(1, 1, 0), base_orientation=(0, 0, 0, 1))
objs = DynamicObject(item=robot, trajectory=traj)

env = Simple2ArmEnv(objects=[objs])
num_samples = 1000
max_num_samples = 2000

In [ ]:
# visualize environment
%matplotlib inline
import matplotlib.pyplot as plt
env.load()
plt.imshow(env.render())
plt.show()

In [ ]:
from planner.learned.GNN_dynamic_planner import GNNDynamicPlanner

In [ ]:
weights_path = ['data/weights/dynamic/2arms/weights_gnn.pt','data/weights/dynamic/2arms/weights_head.pt']

In [ ]:
start, goal = np.array([0.] * 2), np.array([np.pi*7/8, 0])
# start, goal = np.array([0.] * 2), np.array([0, 0])
seed_everything(42)
model_args = dict(config_size=env.robot.config_dim, embed_size=32, obs_size=9)
planner = GNNDynamicPlanner(num_samples=num_samples, max_num_samples=max_num_samples, model_args=model_args, use_bctc=False, stop_when_success=True)
# load trained weights
planner.load_model(weights_path)
result = planner.plan(env, start, goal, timeout=('time', 1000))
if result.solution:
    print('success', result)
else:
    print('fail')